In [ ]:
import pandas as pd
import mxnet as mx
from mxnet import nd, autograd, gluon

In [ ]:
data_ctx = mx.cpu()
model_ctx = mx.cpu()

In [ ]:
batch_size = 100
simdata = pd.read_csv("data/trainingdata.csv")

num_train_samples = 70000
num_test_samples = 30000

simtrain = simdata.sample(n=num_train_samples)
simtest = simdata.drop(simtrain.index).sample(num_test_samples)

predictors = ["square1", "square2","square3", "square4","square5", "square6","square7", "square8", "TrapSpacing"]

Xtrain = nd.array(simtrain[predictors], ctx=data_ctx)
Ytrain = nd.array(simtrain["Density"], ctx=data_ctx)

Xtest = nd.array(simtest[predictors], ctx=data_ctx)
Ytest = nd.array(simtest["Density"], ctx=data_ctx)

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtrain, Ytrain), batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtest, Ytest), batch_size=batch_size, shuffle=True)

In [ ]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(1))
    net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)

In [ ]:
net

In [ ]:
square_loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})

In [ ]:
epochs = 500
loss_train_sequence = []
loss_test_sequence = []

for e in range(epochs):
    # train epoch loop
    cumulative_train_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_train_loss += nd.sum(loss).asscalar()
    loss_train_sequence.append(cumulative_train_loss/num_train_samples)
    
#     print("Epoch %s, train loss: %s" % (e, cumulative_train_loss/num_train_samples))

    # test epoch loss loop
    cumulative_test_loss = 0
    for i, (data, label) in enumerate(test_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        output = net(data)
        loss = square_loss(output, label)
        cumulative_test_loss += nd.sum(loss).asscalar()
    loss_test_sequence.append(cumulative_test_loss/num_test_samples)
    
    print("Epoch %s, train loss: %s, test loss: %s" % (e, cumulative_train_loss/num_train_samples, cumulative_test_loss/num_test_samples))

In [ ]:
# plot the convergence of the estimated loss function
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

plt.figure(num=None,figsize=(8, 6))
plt.plot(loss_train_sequence)
plt.plot(loss_test_sequence)

# Adding some bells and whistles to the plot
plt.grid(True, which="both")
plt.xlabel('epoch',fontsize=14)
plt.ylabel('average loss',fontsize=14)

In [ ]:
net.summary(nd.random.uniform(shape=(8,9)))

In [ ]:
square_loss(net(Xtrain), Ytrain).mean()

In [ ]:
square_loss(net(Xtest), Ytest).mean()

In [ ]:
plt.hist((Ytest.flatten()-net(Xtest)).asnumpy())